# Modelos Squence-to-Sequence

Las redes neuronales recurrentes se pueden utilizar para modelar el lenguaje, como ya vimos en la otra libreta RNN. Esto plantea una pregunta interesante: ¿podriamos condicionar las palabras generadas en alguna entrada y generar una respuesta significativa? Por ejemplo, podriamos entrenar una red neuronal para traducir del Ingles al Frances? Resulta que la respuesta es sí.

Este tutorial mostrara como construir y entrenar dicho sistema. Estamos asumiendo que ya ha instalado a traves del paquete de pip, hemos clonado el repositorio git tensorflow, y estan en la raiz del arbol git.

A continuacion, puede iniciar ejecutando el programa de traducir:

 **cd tensorflow/models/rnn/translate python translate.py --data_dir [your_data_directory] **
 
Se descargara los datos de traduccion-Ingles-Frances, prepararlo para el entrenamiento y la prueba. Toma aproximadamente 20 GB de espacio en disco, y un tiempo para descargar y preparar.

Este tutorial hace referencia a los siguientes archivos de **models/rnn**: 

* seq2seq.py: Es la libreria para construir el modelo Sequence-to-Sequence.
* translate/seq2seq_model.py: Es la traduccion neuronal para el modelo Sequence-to-Sequence.
* translate/data_utils.py: Son funciones de ayuda para preparar la traduccion de datos.
* translate/translate.py: Entrena y corre el modelo de traduccion.

## Fundamentos Sequence-to-Sequence

Un modelo basico de Sequence-to-Sequence consta de dos redes neuronales recurrentes (RNNs): un codificador que procesa la entrada y un decodificador que genera la salida. Esta arquitectura básica se representa a continuación.

![](https://www.tensorflow.org/versions/master/images/basic_seq2seq.png)

Cada caja en la imagen de arriba representa una celda de la RNN.El Codificador y decodificador pueden compartir pesos o, como es más frecuente, utilizar diferente de parámetros.

En el modelo basico se muestra arriba, cada entrada tiene que ser codificado en un vector de estado de tamaño fijo, ya que es la única cosa que se pasa al decodificador. Una red sequence-to-sequence multi-capa con celdas LSTM se parece a esto.

![](https://www.tensorflow.org/versions/master/images/attention_seq2seq.png)

## Libreria TensorFlow seq2seq

Como se puede ver arriba, hay muchos diferentes modelos de sequence-tosequence. Cada uno de estos modelos puede utilizar diferentes celdas RNN, pero todas ellas aceptan entradas de codificador y decodificador. El modelo basico RNN codificador-decodificador sequence-to-sequence funciona como sigue.

In [ ]:
outputs, states = basic_rnn_seq2seq(encoder_inputs, decoder_inputs, cell)

En **encoder_inputs** es una lista de los tensors que representan entradas al codificador, es decir, que corresponden a las letras A, B, C en la primera imagen de arriba. Del mismo modo, **decoder_inputs** son tensors que representan entradas al decodificador, GO, W, X, Y, Z en la primera imagen.

El argumento **cell** es una instancia de la clase **models.rnn.rnn_cell.RNNCell** que determina que celdas se utilizaran dentro del modelo.

La llamada a **basic_rnn_seq2seq** devuelve dos argumentos: **outputs** y **states**. Ambos son listas de los tensors de la misma longitud que **decoder_inputs** Naturalmente, **outputs** corresponden a las salidas del decodificador en cada paso de tiempo, en la primera imagen de arriba que sería W, X, Y, Z, EOS.

En muchas aplicaciones de modelos de sequence-to-sequence, la salida del decodificador en el tiempo t se realimenta y se convierte en la entrada del decodificador en el tiempo t + 1. Durante el tiempo de prueba, cuando se decodifica una secuencia, así es como se construye la secuencia. Durante el entrenamiento, por otro lado, es común proporcionar la entrada correcta al decodificador en cada paso de tiempo, incluso si el decodificador ha cometido un error antes. Funciones en **seq2seq.py** soportan ambos modos utilizando el argumento **feed_previous**. Por ejemplo, vamos a analizar el siguiente uso de un modelo de RNN.

In [ ]:
outputs, states = embedding_rnn_seq2seq(
    encoder_inputs, decoder_inputs, cell,
    num_encoder_symbols, num_decoder_symbols,
    output_projection=None, feed_previous=False)

En el modelo **embedding_rnn_seq2seq**, todas las entradas (tanto **encoder_inputs** y **decoder_inputs**) son enteros-tensor que representan valores discretos. 

En la invocación anterior, pusimos **feed_previous** a False. Esto significa que el decodificador utilizará los tensors de **decoder_inputs**. Si establecemos **feed_previous** en True, el decodificador sólo usa el primer elemento de **decoder_inputs**. El resto de los tensore de esta lista serian ignorados, y en lugar la salida anterior del codificador se utilizaría.

Un argumento importante usado anteriormente es **output_projection**. Si no se especifica, la salida del modelo de incorporación serán tensors de forma lotes de tamaño por **num_decoder_symbols** ya que representan los logits para cada símbolo generado. Cuando entrenamos modelos con grandes vocabularios de salida, por ejemplo, cuando **num_decoder_symbols** es grande, no es práctico almacenar esto. En cambio, es mejor devolver tensors de salida más pequeños, que posteriormente serán proyectadas en una gran tensor de salida usando **output_projection**.

Además de **basic_rnn_seq2seq** y **embedding_rnn_seq2seq** hay algunos más modelos sequence-to-sequence en **seq2seq.py**.

# Neural Modelo Traducción

Mientras el núcleo del modelo sequence-to-sequence se construye mediante las funciones en **models/rnn/seq2seq.py** hay algunos trucos que vale la pena mencionar que se utilizan en nuestro modelo de traducción en **models/rnn/translate/seq2seq_model.py**.

## Softmax muestreado y proyección de salida

Por un lado, como ya se ha mencionado anteriormente, queremos utilizar softmax muestreado para manejar el vocabulario de salida grande. Para decodificar de ella, tenemos que hacer un seguimiento de la proyección de salida. Tanto la pérdida softmax muestreado y las proyecciones de salida se construyen mediante el siguiente código en **seq2seq_model.py**.

In [ ]:
if num_samples > 0 and num_samples < self.target_vocab_size:
    w = tf.get_variable("proj_w", [size, self.target_vocab_size])
    w_t = tf.transpose(w)
    b = tf.get_variable("proj_b", [self.target_vocab_size])
    output_projection = (w, b)

    def sampled_loss(inputs, labels):
      labels = tf.reshape(labels, [-1, 1])
      return tf.nn.sampled_softmax_loss(w_t, b, inputs, labels, num_samples,
                                        self.target_vocab_size)

Primero, nota que sólo se construye un softmax muestreado si el número de muestras (512 por defecto) es menor que el tamaño del vocabulario objetivo. Para vocabularios más pequeños que 512 podría ser una mejor idea utilizar sólo un standard softmax loss.

Entonces, como puedes ver, se construye una proyección de salida. Es un par, que consiste en una matriz de peso y un vector de bias. Si se utiliza, la celda RNN devolvera vectores de forma por lotes de tamaño por **size** en lugar de lotes de tamaño por **target_vocab_size** Para recuperar logits, tenemos que multiplicar por la matriz de peso y añadir los biases, como se hace en las líneas 124-126 en **seq2seq_model.py**

In [ ]:
if output_projection is not None:
  self.outputs[b] = [tf.matmul(output, output_projection[0]) +
                     output_projection[1] for ...]

## Bucketing y Paddin

Además del softmax muestreado, nuestro modelo de traducción también hace uso de bucketing, que es un método para manejar eficientemente oraciones de diferentes longitudes. Al traducir Ingles al Frances, tendremos oraciones en ingles de diferentes longitudes L1 en la entrada y oraciones francesas de diferentes longitudes L2 en la salida. Desde que la sentencia de Inglés pasa como **encoder_inputs** y la sentencia francesa viene como **decoder_inputs**, debemos crear en principio un modelo seq2seq para cada par (L1, L2 + 1) de longitudes de una oracion en Frances e Ingles. Esto resultara en un enorme grafo que consta de muchos subgraphos muy similares. Por otro lado, podríamos simplemente pad cada oracion con un símbolo especial PAD. Entonces necesitaríamos sólo un modelo seq2seq, para las longitudes padded. Pero en oraciones más corta nuestro modelo sería ineficiente, la codificación y decodificación de muchos símbolos PAD que son inútiles.

Como un compromiso entre construir un grafo para cada par de longitudes y padding de una sola longitud, se utiliza un número de buckets y pad en cada oracion a la longitud del bucket por encima de ella. En **translate.py** utilizamos los siguientes buckets por defecto.

In [ ]:
buckets = [(5, 10), (10, 15), (20, 25), (40, 50)]

Esto significa que si la entrada es una oracion en Ingles con 3 tokens, y la salida correspondiente es una oracion francesa con 6 tokens, entonces van a ser puestos en el primer bucket y padded a la longitud 5 para las entradas del codificador, y la longitud 10 para las entradas del decodificador. Si tenemos una oracion en Ingles con 8 fichas y la oracion francesa correspondiente cuenta con 18 fichas, entonces no van a caber en el bucket(10, 15), por lo que el bucket(20, 25) se utilizaria.

Recuerda que en la construcción de las entradas del decodificador pretendemos el simbolo especial **GO** para los datos de entrada. Esto se hace en la funcion **get_batch()** de **seq2seq_model.py** que también invierte la  oracion de entrada en Ingles. Para ponerlo todo junto, imagina que tenemos la frase "I go.", en tokens como ["I", "go", "."] como entrada y la frase" Je vais ". como salida, tokens ["Je", "vais", "."] Se puso en el bucket(5, 10), con entradas de encoder que representan [PAD PAD "." "go" "I"] y las entradas del decodificador [GO "Je" "vais" "." EOS PAD PAD PAD PAD PAD].

# Vamos a ejecutarlo

Para entrenar el modelo descrito anteriormente, necesitamos un gran corpus Ingles-Frances. Vamos a utilizar el corpus 10 ^ 9-Francés-Inglés del Sitio [Web WMT'15](http://www.statmt.org/wmt15/translation-task.html) para el entrenamiento, y la prueba de noticias 2013 desde el mismo sitio que desarrollo el conjunto. Ambos conjuntos de datos se descargarán en **data_dir**.

>python translate.py
  --data_dir [your_data_directory] --train_dir [checkpoints_directory]
  --en_vocab_size=40000 --fr_vocab_size=40000
  
Toma aproximadamente 18GB de espacio en disco y varias horas para entrenar el corpus. Es desempaquetado, archivos de vocabulario son creados en **data_dir** y luego el corpus es tokenizado y convertido a entero id. Tenga en cuenta los parámetros que determinan tamaños de vocabulario. En el ejemplo anterior, todas las palabras fuera de los 40K más comunes se convertirán en un UNK token que representa las palabras desconocidas. Así que si usted cambia el tamaño del vocabulario, el binario se volverá a asignar el corpus de Token-ids de nuevo.

Después de preparar los datos, El entrenamiento comienza. Los parámetros por defecto de **translate** se establecen en valores muy grandes. Los modelos grandes entrenados durante un largo tiempo dan buenos resultados, pero podría tomar mucho tiempo o use demasiada memoria para su GPU. Usted puede solicitar entrenar con un modelo más pequeño como en el siguiente ejemplo.


>python translate.py
  --data_dir [your_data_directory] --train_dir [checkpoints_directory]
  --size=256 --num_layers=2 --steps_per_checkpoint=50
  
El comando anterior va entrenar un modelo con 2 capas (el valor predeterminado es 3), cada capa con 256 unidades (por defecto es 1024), y guardara un checkpoint cada 50 pasos (el valor predeterminado es 200). Puedes jugar con estos parámetros para saber qué tan grande puede ser un modelo para encajar en la memoria de la GPU.

Durante el entrenamiento, cada **steps_per_checkpoint**  pasos se imprimira estadísticas de las medidas recientes. Con los parámetros por defecto (3 capas de tamaño 1024), los primeros mensajes de este aspecto.

>global step 200 learning rate 0.5000 step-time 1.39 perplexity 1720.62
  eval: bucket 0 perplexity 184.97
  eval: bucket 1 perplexity 248.81
  eval: bucket 2 perplexity 341.64
  eval: bucket 3 perplexity 469.04
global step 400 learning rate 0.5000 step-time 1.38 perplexity 379.89
  eval: bucket 0 perplexity 151.32
  eval: bucket 1 perplexity 190.36
  eval: bucket 2 perplexity 227.46
  eval: bucket 3 perplexity 238.66
  
Se puede ver que cada paso tiene poco menos de 1.4 segundos, la perplejidad en el conjunto de entrenamiento y las perplejidades sobre el conjunto de desarrollo para cada bucket. Después de cerca de 30k pasos, vemos perplejidades en oraciones cortas (bucket 0 y 1) que van a un solo dígito. Puesto que el corpus de entrenamiento contiene ~ 22M oraciones, un epoch se tarda unos 340K pasos con lotes de tamaño de los 64. En este punto, el modelo se puede utilizar para traducir frases en inglés a francés utilizando la opcion **--decode**.

>python translate.py --decode
  --data_dir [your_data_directory] --train_dir [checkpoints_directory]

>Reading model parameters from /tmp/translate.ckpt-340000
>  Who is the president of the United States?
 Qui est le président des États-Unis ?